# Plot3D Read/Write File Formats

This tutorial demonstrates the different file format options for reading and writing Plot3D mesh files.

## Supported Formats

| Format | Description | Use Case |
|--------|-------------|----------|
| Binary (default) | Compact binary format | General use, fastest I/O |
| ASCII | Human-readable text | Debugging, small meshes |
| Fortran | Fortran unformatted with record markers | Legacy CFD codes |
| Big-endian | Big-endian byte order | Cross-platform compatibility |
| Single precision | 32-bit floats | Reduced file size |

## Install Dependencies

In [ ]:
!pip install plot3d -q

## Import Libraries

In [ ]:
import numpy as np
import os
from plot3d import Block, read_plot3D, write_plot3D

## Create Sample Mesh

Create a simple unit cube mesh for demonstration.

In [ ]:
# Create a 10x10x10 unit cube mesh
ni, nj, nk = 10, 10, 10
x = np.linspace(0, 1, ni)
y = np.linspace(0, 1, nj)
z = np.linspace(0, 1, nk)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

block = Block(X=X, Y=Y, Z=Z)
print(f"Created block: {block.IMAX} x {block.JMAX} x {block.KMAX}")

## Binary Format (Default)

The default format is binary with little-endian byte order and double precision.

In [ ]:
# Write binary (default settings)
write_plot3D('test_binary.xyz', [block])

# Read back and verify
blocks_read = read_plot3D('test_binary.xyz')
print(f"Data matches: {np.allclose(block.X, blocks_read[0].X)}")

## ASCII Format

Human-readable format using scientific notation. Set `binary=False`.

In [ ]:
# Write ASCII
write_plot3D('test_ascii.xyz', [block], binary=False)

# Read back
blocks_ascii = read_plot3D('test_ascii.xyz', binary=False)
print(f"Data matches: {np.allclose(block.X, blocks_ascii[0].X)}")

# Show first few lines
print("\nFile contents (first 5 lines):")
with open('test_ascii.xyz') as f:
    for i, line in enumerate(f):
        if i < 5: print(line.rstrip())

## Single Precision

Use 32-bit floats for smaller files. Set `double_precision=False` when writing and `read_double=False` when reading.

In [ ]:
# Write single precision
write_plot3D('test_single.xyz', [block], double_precision=False)

# Read back (must specify read_double=False)
blocks_single = read_plot3D('test_single.xyz', read_double=False)
print(f"Data matches: {np.allclose(block.X, blocks_single[0].X, rtol=1e-6)}")

# Compare sizes
print(f"\nDouble: {os.path.getsize('test_binary.xyz')} bytes")
print(f"Single: {os.path.getsize('test_single.xyz')} bytes")

## Big-Endian Byte Order

For compatibility with big-endian systems, set `big_endian=True`.

In [ ]:
# Write big-endian
write_plot3D('test_big_endian.xyz', [block], big_endian=True)

# Read back (must specify big_endian=True)
blocks_be = read_plot3D('test_big_endian.xyz', big_endian=True)
print(f"Data matches: {np.allclose(block.X, blocks_be[0].X)}")

## Fortran Unformatted Binary

Many legacy CFD codes (written in Fortran) use "unformatted" binary files with record markers. Set `fortran=True`.

The record structure is:
```
[4-byte record length][data][4-byte record length]
```

In [ ]:
# Write Fortran format
write_plot3D('test_fortran.xyz', [block], fortran=True)

# Read back (must specify fortran=True)
blocks_fortran = read_plot3D('test_fortran.xyz', fortran=True)
print(f"Data matches: {np.allclose(block.X, blocks_fortran[0].X)}")

# Fortran files have overhead from record markers
print(f"\nC binary:       {os.path.getsize('test_binary.xyz')} bytes")
print(f"Fortran binary: {os.path.getsize('test_fortran.xyz')} bytes")

## Combining Options

Options can be combined as needed.

In [ ]:
# Fortran format with single precision
write_plot3D('test_fortran_single.xyz', [block], fortran=True, double_precision=False)
blocks_fs = read_plot3D('test_fortran_single.xyz', fortran=True, read_double=False)
print(f"Fortran + Single precision matches: {np.allclose(block.X, blocks_fs[0].X, rtol=1e-6)}")

# ASCII with single precision formatting
write_plot3D('test_ascii_single.xyz', [block], binary=False, double_precision=False)
blocks_as = read_plot3D('test_ascii_single.xyz', binary=False)
print(f"ASCII + Single precision matches: {np.allclose(block.X, blocks_as[0].X, rtol=1e-6)}")

## API Reference

### write_plot3D
```python
write_plot3D(
    filename: str,              # Output file path
    blocks: List[Block],        # List of Block objects
    binary: bool = True,        # Binary (True) or ASCII (False)
    big_endian: bool = False,   # Little-endian (False) or big-endian (True)
    double_precision: bool = True,  # Double (True) or single (False) precision
    fortran: bool = False,      # Standard (False) or Fortran unformatted (True)
    batch_size: int = 100       # Write buffer size
)
```

### read_plot3D
```python
read_plot3D(
    filename: str,              # Input file path
    binary: bool = True,        # Binary (True) or ASCII (False)
    big_endian: bool = False,   # Little-endian (False) or big-endian (True)
    read_double: bool = True,   # Double (True) or single (False) precision
    fortran: bool = False       # Standard (False) or Fortran unformatted (True)
) -> List[Block]
```

## Cleanup

In [ ]:
# Remove test files
for f in ['test_binary.xyz', 'test_ascii.xyz', 'test_single.xyz', 
          'test_big_endian.xyz', 'test_fortran.xyz', 
          'test_fortran_single.xyz', 'test_ascii_single.xyz']:
    if os.path.exists(f):
        os.remove(f)
print("Test files cleaned up.")